# Nano Banana (Gemini 2.5 Flash Image) – Real-World Use Cases Notebook

This hands-on notebook shows how to build real applications with Gemini 2.5 Flash Image (aka Nano Banana): text-to-image, image editing, multi-image composition, batch pipelines, and lightweight evaluation.

Highlights:
- Uses the Google GenAI SDK (python package: google-genai).
- Model: "gemini-2.5-flash-image-preview" for generation and editing. Outputs include SynthID watermarking by default.
- Scenarios: e-commerce background standardization, heritage photo restoration, marketing banners, game asset previsualization.

Note: You need a Gemini API key from Google AI Studio. Set GEMINI_API_KEY as an environment variable before running.

## 0) Setup – Install and Authenticate

In [ ]:
!pip -q install -U google-genai pillow numpy scikit-image

In [ ]:
import os
from io import BytesIO
from PIL import Image
import numpy as np
from skimage.metrics import structural_similarity as ssim

from google import genai
from google.genai import types

# Ensure API key is present in environment.
assert os.environ.get("GEMINI_API_KEY"), "Please set GEMINI_API_KEY in your environment."

client = genai.Client()
MODEL = "gemini-2.5-flash-image-preview"
print("Client initialized. Using model:", MODEL)

## 1) Text to Image – Prompt Patterns
Goal: Generate consistent, high-quality images using camera and lighting descriptors when photorealism is desired.

In [ ]:
from IPython.display import display

prompt_photo = (
    "A photorealistic product shot of a ceramic coffee mug on a wooden table, "
    "morning window light, soft shadows, 50mm lens shallow depth of field, "
    "crisp highlights on the rim, 3:2 aspect ratio"
)

resp = client.models.generate_content(
    model=MODEL,
    contents=[prompt_photo]
)

img_bytes = None
if resp and resp.candidates:
    for part in resp.candidates[0].content.parts:
        if getattr(part, "inline_data", None):
            img_bytes = part.inline_data.data
            break

if img_bytes:
    img = Image.open(BytesIO(img_bytes))
    img.save("t2i_product_shot.png")
    display(img)
else:
    print("No image bytes returned. Check quota, keys, or prompt.")

Prompt template cheatsheet:
- Photorealistic: shot type, subject, environment, lighting, mood, camera/lens, aspect ratio.
- Stylized/Illustration: style (watercolor, cel-shaded, oil), palette, brushwork, texture, composition.
- Logos/Text: vector style, kerning, alignment, background contrast, negative space.
Experiment: Tweak one descriptor at a time to observe the effect on outputs.

## 2) Image plus Text to Edited Image – Background Replacement
Goal: Replace background of an existing product image for consistent catalog presentation.
Input: set INPUT_PATH to an existing image file (jpg or png).

In [ ]:
from pathlib import Path

INPUT_PATH = "input_product.jpg"  # Replace with your file path
assert Path(INPUT_PATH).exists(), f"Provide an input image at {INPUT_PATH}"

image = Image.open(INPUT_PATH).convert("RGB")

edit_prompt = (
    "Replace the background with a clean, matte light-gray studio backdrop, "
    "keep the product edges crisp, no reflections, maintain natural soft shadow"
)

resp_edit = client.models.generate_content(
    model=MODEL,
    contents=[edit_prompt, image]
)

edited_bytes = None
if resp_edit and resp_edit.candidates:
    for part in resp_edit.candidates[0].content.parts:
        if getattr(part, "inline_data", None):
            edited_bytes = part.inline_data.data
            break

if edited_bytes:
    edited_img = Image.open(BytesIO(edited_bytes))
    edited_img.save("edited_product_gray_bg.png")
    display(edited_img)
else:
    print("No edited image returned.")

## 3) Multi-Image Composition – Style Reference and Layout Merge
Goal: Compose a lifestyle scene using a product photo and a brand style reference.
Inputs: product.png and style_ref.png (both must exist).

In [ ]:
PRODUCT = "product.png"
STYLE = "style_ref.png"
assert Path(PRODUCT).exists() and Path(STYLE).exists(), "Add PRODUCT and STYLE images."

img_product = Image.open(PRODUCT).convert("RGB")
img_style = Image.open(STYLE).convert("RGB")

compose_prompt = (
    "Compose a lifestyle photo on a wooden kitchen counter, soft window light. "
    "Use the color palette from the style reference, keep product scale natural, "
    "subtle depth of field, premium brand feel."
)

resp_comp = client.models.generate_content(
    model=MODEL,
    contents=[compose_prompt, img_product, img_style]
)

comp_bytes = None
if resp_comp and resp_comp.candidates:
    for part in resp_comp.candidates[0].content.parts:
        if getattr(part, "inline_data", None):
            comp_bytes = part.inline_data.data
            break

if comp_bytes:
    comp_img = Image.open(BytesIO(comp_bytes))
    comp_img.save("composed_lifestyle.png")
    display(comp_img)
else:
    print("No composition image returned.")

## 4) Iterative Refinement – Conversational Edits
You can chain edits by passing the previous output with a new prompt to simulate a designer feedback loop.

In [ ]:
BASE_EDIT = "composed_lifestyle.png"
assert Path(BASE_EDIT).exists(), "Run the previous cell to create composed_lifestyle.png"

base_img = Image.open(BASE_EDIT).convert("RGB")
refine_prompt = (
    "Warm up the color temperature slightly, add soft morning sun beams, "
    "increase contrast subtly, remove any harsh reflections"
)

resp_refine = client.models.generate_content(
    model=MODEL,
    contents=[refine_prompt, base_img]
)

refined_bytes = None
if resp_refine and resp_refine.candidates:
    for part in resp_refine.candidates[0].content.parts:
        if getattr(part, "inline_data", None):
            refined_bytes = part.inline_data.data
            break

if refined_bytes:
    refined_img = Image.open(BytesIO(refined_bytes))
    refined_img.save("composed_lifestyle_refined.png")
    display(refined_img)
else:
    print("No refined image returned.")

## 5) Quick Evaluation – SSIM (Structural Similarity)
For editing workflows, compare before and after to ensure key details are preserved. SSIM in [0,1], higher is closer.

In [ ]:
def to_gray(img):
    return np.array(img.convert("L"), dtype=np.float32) / 255.0

orig = Image.open(INPUT_PATH).convert("RGB")
edited = Image.open("edited_product_gray_bg.png").convert("RGB")

score, _ = ssim(to_gray(orig), to_gray(edited), full=True)
print(f"SSIM (original vs edited): {score:.3f}")
display(orig, edited)

# Real-World Use Case PipelinesBelow are four production-style mini-pipelines you can adapt.

## Use Case A) E-commerce – Unified Backgrounds for Catalog
Goal: Standardize product backgrounds (light gray or white) and generate a lifestyle alternative for PDP or ads.
Inputs: folder of product photos in ecom_in/. Outputs go to ecom_out/.

In [ ]:
import glob
from pathlib import Path

in_dir = Path("ecom_in")
out_dir = Path("ecom_out")
out_dir.mkdir(exist_ok=True)

bg_prompt = (
    "Replace the background with a clean white sweep, keep natural product shadow, "
    "no clipping, preserve edges, center the product, PDP-ready"
)

lifestyle_prompt = (
    "Create a subtle lifestyle setting: matte wooden shelf, soft daylight, muted premium palette, realistic scale"
)

paths = glob.glob(str(in_dir / "*.jpg")) + glob.glob(str(in_dir / "*.png"))
for path in paths:
    img = Image.open(path).convert("RGB")

    resp_bg = client.models.generate_content(model=MODEL, contents=[bg_prompt, img])
    bg_bytes = None
    if resp_bg and resp_bg.candidates:
        for p in resp_bg.candidates[0].content.parts:
            if getattr(p, "inline_data", None):
                bg_bytes = p.inline_data.data
                break
    if bg_bytes:
        out_bg = Image.open(BytesIO(bg_bytes))
        out_bg.save(out_dir / (Path(path).stem + "_bg.png"))

    resp_ls = client.models.generate_content(model=MODEL, contents=[lifestyle_prompt, img])
    ls_bytes = None
    if resp_ls and resp_ls.candidates:
        for p in resp_ls.candidates[0].content.parts:
            if getattr(p, "inline_data", None):
                ls_bytes = p.inline_data.data
                break
    if ls_bytes:
        out_ls = Image.open(BytesIO(ls_bytes))
        out_ls.save(out_dir / (Path(path).stem + "_lifestyle.png"))

print("E-commerce batch complete -> ecom_out/")

## Use Case B) Photo Restoration – Heritage and Archives
Goal: Denoise, remove scratches, gentle color correction, preserve identity.
Input: heritage_in/damaged_*.png

In [ ]:
in_dir = Path("heritage_in")
out_dir = Path("heritage_out")
out_dir.mkdir(exist_ok=True)

restore_prompt = (
    "Restore the photograph: remove scratches, reduce film grain, recover details, "
    "gentle color correction, preserve facial identity, no beautification"
)

paths = glob.glob(str(in_dir / "damaged_*.png"))
for path in paths:
    img = Image.open(path).convert("RGB")
    resp_r = client.models.generate_content(model=MODEL, contents=[restore_prompt, img])
    r_bytes = None
    if resp_r and resp_r.candidates:
        for p in resp_r.candidates[0].content.parts:
            if getattr(p, "inline_data", None):
                r_bytes = p.inline_data.data
                break
    if r_bytes:
        out = Image.open(BytesIO(r_bytes))
        out.save(out_dir / (Path(path).stem + "_restored.png"))

print("Heritage restoration batch complete -> heritage_out/")

## Use Case C) Marketing – Banner and Social Variants
Goal: Generate banner backgrounds and produce multiple crops/aspect ratios for social platforms.
We generate a base key visual, then create 1:1, 16:9, and 9:16 variants.

In [ ]:
kv_prompt = (
    "Design a clean tech banner background with gradient blues and purples, "
    "soft bokeh lights, diagonal energy lines, ample negative space top-left for a headline and logo"
)
resp_kv = client.models.generate_content(model=MODEL, contents=[kv_prompt])
kv_bytes = None
if resp_kv and resp_kv.candidates:
    for p in resp_kv.candidates[0].content.parts:
        if getattr(p, "inline_data", None):
            kv_bytes = p.inline_data.data
            break
if kv_bytes:
    kv_img = Image.open(BytesIO(kv_bytes)).convert("RGB")
    kv_img.save("marketing_kv_base.png")
    display(kv_img)

    def resize_exact(img, w, h):
        return img.resize((w, h), Image.LANCZOS)

    resize_exact(kv_img, 1200, 1200).save("marketing_kv_square_1200.png")
    resize_exact(kv_img, 1600, 900).save("marketing_kv_16x9_1600x900.png")
    resize_exact(kv_img, 1080, 1920).save("marketing_kv_9x16_1080x1920.png")
else:
    print("KV generation failed.")

## Use Case D) Game and Simulation – Fast Asset Previsualization
Goal: Generate concept-art variants for props with consistent theme and viewpoints.

In [ ]:
theme = "retro-futuristic space diner"
prop = "coffee machine"

variants = [
    f"Concept art, {prop}, {theme}, chrome accents, neon highlights, ambient occlusion, orthographic view",
    f"Concept art, {prop}, {theme}, pastel palette, soft reflections, exploded view schematic",
    f"Concept art, {prop}, {theme}, dark mode, rim lighting, top-down view"
]

for i, ptxt in enumerate(variants, 1):
    r = client.models.generate_content(model=MODEL, contents=[ptxt])
    b = None
    if r and r.candidates:
        for prt in r.candidates[0].content.parts:
            if getattr(prt, "inline_data", None):
                b = prt.inline_data.data
                break
    if b:
        Image.open(BytesIO(b)).save(f"asset_previz_{i}.png")
print("Game asset concept variants saved: asset_previz_*.png")

# Prompt Engineering Patterns
1. Describe the scene in a sentence or two; avoid isolated keywords.
2. For photorealism, add camera or lens details and lighting.
3. For edits, be explicit about what to preserve (edges, logos, faces) and what to change.
4. Iterate with small, targeted refinements using the previous output.
5. Maintain brand consistency via palettes, composition rules, and negative space conventions.

# Batch API Notes and Production Tips
- Prefer batching and idempotent filenames in pipelines.
- Keep a manifest (CSV or JSON) with input to output mappings for traceability.
- Add lightweight QA (for example, SSIM for edit flows; human spot checks for brand compliance).
- Respect licensing and privacy. Generated images include SynthID watermarking for provenance.